# 1. Insert countries data into postgresql

In [19]:
import pandas as pd

cs = pd.read_csv('data/countries.csv')
eur_s = pd.read_csv('data/european_seasons.csv')
s = pd.read_csv('data/seasons.csv')

In [20]:
cs.head()

,name,code,logo
0,Albania,AL,https://media.api-sports.io/flags/al.svg
1,Algeria,DZ,https://media.api-sports.io/flags/dz.svg
2,Andorra,AD,https://media.api-sports.io/flags/ad.svg
3,Angola,AO,https://media.api-sports.io/flags/ao.svg
4,Antigua-And-Barbuda,AG,https://media.api-sports.io/flags/ag.svg


In [21]:
import psycopg2

db_params = {
    'host': 'localhost',
    'database': 'preds',
    'user': 'postgres',
    'password': 'pass',
    'port': '5432'
}

try:
    conn = None
    cur = None 

    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()

    insert_query = """
        INSERT INTO {} ({})
        VALUES ({})
    """.format('countries', ','.join(cs.columns), ','.join(['%s']*len(cs.columns)))

    cur.executemany(insert_query, cs.values.tolist())
    print('inserted')
    conn.commit()
except Exception as e:
    print(f'Error {e}')

finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

inserted


##### changed foreign key

In [22]:
import psycopg2

db_params = {
    'host': 'localhost',
    'database': 'preds',
    'user': 'postgres',
    'password': 'pass',
    'port': '5432'
}

try:
    conn = None
    cur = None 

    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()

    drop_constraint_query = """
    ALTER TABLE teams
    DROP CONSTRAINT IF EXISTS fk_team_country;
    """
    cur.execute(drop_constraint_query)
    conn.commit()
    print('Dropped existing foreign key constraint')
    
    
    # Step 2: Add the new foreign key constraint
    add_constraint_query = """
    ALTER TABLE teams
    ADD CONSTRAINT fk_team_country
    FOREIGN KEY (team_country) REFERENCES countries(name);
    """
    cur.execute(add_constraint_query)
    conn.commit()
    print('Added new foreign key constraint')
    conn.commit()
    

except Exception as e:
    print(f'Error {e}')

finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()


Dropped existing foreign key constraint
Added new foreign key constraint


##### change col type

In [3]:
import psycopg2

db_params = {
    'host': 'localhost',
    'database': 'preds',
    'user': 'postgres',
    'password': 'pass',
    'port': '5432'
}

try:
    conn = None
    cur = None 

    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()

    drop_constraint_query = """
    ALTER TABLE fixtures_updated
    ALTER COLUMN teams_home_winner TYPE INT USING teams_home_winner::integer,
    ALTER COLUMN teams_away_winner TYPE INT USING teams_away_winner::integer;
    """
    cur.execute(drop_constraint_query)
    conn.commit()
    print('changed col type')

except Exception as e:
    print(f'Error {e}')

finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()


changed col type


# Insert league data to postgresql

In [23]:
c = pd.read_csv('data/contests.csv')
c.head()

,league_id,name,type,logo,country,country_code,seasons
0,4,Euro Championship,Cup,https://media.api-sports.io/football/leagues/4...,World,NaN,5
1,21,Confederations Cup,Cup,https://media.api-sports.io/football/leagues/2...,World,NaN,3
2,61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,14
3,144,Jupiler Pro League,League,https://media.api-sports.io/football/leagues/1...,Belgium,BE,14
4,71,Serie A,League,https://media.api-sports.io/football/leagues/7...,Brazil,BR,15


In [24]:
import psycopg2

db_params = {
    'host': 'localhost',
    'database': 'preds',
    'user': 'postgres',
    'password': 'pass',
    'port': '5432'
}

try:
    conn = None
    cur = None 

    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()

    insert_query = """
        INSERT INTO {} ({})
        VALUES ({})
    """.format('leagues', ','.join(c.columns), ','.join(['%s']*len(c.columns)))

    cur.executemany(insert_query, c.values.tolist())
    print('inserted')
    conn.commit()
except Exception as e:
    print(f'Error {e}')

finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

inserted


# 2. Teams stats in each season

In [12]:
import pandas as pd

c = pd.read_csv('data/contests.csv')
s = pd.read_csv('data/seasons.csv')
es = pd.read_csv('data/european_seasons.csv')

In [11]:
c.head()

,league_id,name,type,logo,country,country_code,seasons
0,4,Euro Championship,Cup,https://media.api-sports.io/football/leagues/4...,World,NaN,5
1,21,Confederations Cup,Cup,https://media.api-sports.io/football/leagues/2...,World,NaN,3
2,61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,14
3,144,Jupiler Pro League,League,https://media.api-sports.io/football/leagues/1...,Belgium,BE,14
4,71,Serie A,League,https://media.api-sports.io/football/leagues/7...,Brazil,BR,15


In [5]:
s[s['current']==True].head()

,Unnamed: 0,league_id,year,start,end,current,events,lineups,statistics_fixtures,statistics_players,standings,players,top_scorers,top_assists,top_cards,injuries,predictions,odds
4,4,4,2024,2024-06-14,2024-06-26,True,False,False,False,False,True,False,False,False,False,False,True,False
7,7,21,2017,2017-06-17,2017-07-02,True,True,True,True,False,False,True,True,True,True,False,True,False
21,21,61,2023,2023-08-11,2024-05-18,True,True,True,True,True,True,True,True,True,True,True,True,True
35,35,144,2023,2023-07-28,2024-05-26,True,True,True,True,True,True,True,True,True,True,False,True,True
50,50,71,2024,2024-04-13,2024-12-08,True,True,True,True,True,True,True,True,True,True,False,True,True


In [16]:
me = es[es['current']==True]

In [19]:
mee = me.merge(s,on=['league_id','year'])

In [22]:
from datetime import date
t = date.today()
t

datetime.date(2024, 6, 9)

In [25]:
mee['end'] = pd.to_datetime(mee['end']).dt.date
mee[(mee['end'] > t)&(mee['statistics_fixtures_x']==True)]

,Unnamed: 0_x,league_id,name,country,year,players_x,statistics_fixtures_x,current_x,predictions_x,odds_x,...,statistics_fixtures_y,statistics_players,standings,players_y,top_scorers,top_assists,top_cards,injuries,predictions_y,odds_y
77,423,142,Primera División Femenina,Spain,2023,True,True,True,True,True,...,True,True,True,True,True,True,True,False,True,True
284,1624,103,Eliteserien,Norway,2024,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
385,2228,113,Allsvenskan,Sweden,2024,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
386,2237,114,Superettan,Sweden,2024,True,True,True,True,True,...,True,True,True,True,True,True,True,False,True,True
475,2722,357,Premier Division,Ireland,2024,True,True,True,True,True,...,True,True,True,True,True,True,True,False,True,True
519,3065,116,Premier League,Belarus,2024,True,True,True,True,True,...,True,False,True,True,True,True,True,False,True,True


# Some operations

In [17]:
me = es.merge(c[['league_id', 'type', 'country']], on = 'league_id')

In [19]:
lis = es['league_id'].unique()

In [26]:
ligi = list(c[(c['type']=='League')&(c['league_id'].isin(list(lis)))&(c['seasons']>1)]['league_id'].unique())

In [29]:
s[(s['odds'])&(s['league_id'].isin(ligi))].drop_duplicates(subset=['league_id'])

,Unnamed: 0,league_id,year,start,end,current,events,lineups,statistics_fixtures,statistics_players,standings,players,top_scorers,top_assists,top_cards,injuries,predictions,odds
21,21,61,2023,2023-08-11,2024-05-18,True,True,True,True,True,True,True,True,True,True,True,True,True
35,35,144,2023,2023-07-28,2024-05-26,True,True,True,True,True,True,True,True,True,True,False,True,True
64,64,39,2023,2023-08-11,2024-05-19,True,True,True,True,True,True,True,True,True,True,True,True,True
78,78,78,2023,2023-08-18,2024-05-27,True,True,True,True,True,True,True,True,True,True,True,True,True
92,92,135,2023,2023-08-19,2024-05-26,True,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5985,5985,1009,2023,2024-05-22,2024-05-25,True,False,False,False,False,False,True,True,True,True,False,True,True
5988,5988,1011,2023,2024-05-11,2024-05-26,True,False,False,False,False,False,True,True,True,True,False,True,True
6033,6033,915,2024,2024-03-16,2024-09-07,True,True,False,False,False,True,False,False,False,False,False,True,True
6071,6071,938,2023,2023-07-21,2024-05-18,True,True,False,False,False,True,True,True,True,True,False,True,True


In [18]:
c[c['type']=='League']

,league_id,name,type,logo,country,country_code,seasons
2,61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,14
3,144,Jupiler Pro League,League,https://media.api-sports.io/football/leagues/1...,Belgium,BE,14
4,71,Serie A,League,https://media.api-sports.io/football/leagues/7...,Brazil,BR,15
5,39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,14
6,78,Bundesliga,League,https://media.api-sports.io/football/leagues/7...,Germany,DE,14
...,...,...,...,...,...,...,...
1101,1103,Premiership Women,League,https://media.api-sports.io/football/leagues/1...,Northern-Ireland,GB,1
1102,1104,Liga 3,League,https://media.api-sports.io/football/leagues/1...,Georgia,GE,1
1104,1106,Carioca C,League,https://media.api-sports.io/football/leagues/1...,Brazil,BR,1
1105,1107,Mineiro U20,League,https://media.api-sports.io/football/leagues/1...,Brazil,BR,1
